In [54]:
import pandas as pd
import numpy as np


data = pd.read_csv('Data/train.csv')
data.head()

test_data = pd.read_csv('Data/test.csv')


In [55]:
data['gender'] = np.where(data['gender']=="Male",1,0)
data['ever_married'] = np.where(data['ever_married']=="Yes",1,0)
data['Residence_type'] = np.where(data['Residence_type']=="Rural",1,0)
data['smoking_status'].fillna(0,inplace=True)
data['smoking_status'] = np.where(data['smoking_status']=="formerly smoked",3,data['smoking_status'])
data['smoking_status'] = np.where(data['smoking_status']=="never smoked",1,data['smoking_status'])
data['smoking_status'] = np.where(data['smoking_status']=="smokes",2,data['smoking_status'])
bmi_mean = data['bmi'].mean()
data['bmi'].fillna(bmi_mean,inplace=True)
data = pd.get_dummies(data,columns = ['work_type'])

In [56]:
test_data['gender'] = np.where(test_data['gender']=="Male",1,0)
test_data['ever_married'] = np.where(test_data['ever_married']=="Yes",1,0)
test_data['Residence_type'] = np.where(test_data['Residence_type']=="Rural",1,0)
test_data['smoking_status'].fillna(0,inplace=True)
test_data['smoking_status'] = np.where(test_data['smoking_status']=="formerly smoked",3,test_data['smoking_status'])
test_data['smoking_status'] = np.where(test_data['smoking_status']=="never smoked",1,test_data['smoking_status'])
test_data['smoking_status'] = np.where(test_data['smoking_status']=="smokes",2,test_data['smoking_status'])
test_data['bmi'].fillna(bmi_mean,inplace=True)
test_data = pd.get_dummies(test_data,columns = ['work_type'])

In [20]:
from sklearn.utils import resample
df_majority = data[data.stroke==0]
df_minority = data[data.stroke==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=int(len(df_majority)/3),    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
data = pd.concat([df_majority, df_minority_upsampled])

In [29]:
train_y = data['stroke']
data.drop(['stroke'], axis = 1, inplace = True)
train_x = data


In [42]:
n_estimator = 40
from sklearn.metrics import auc
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)


X_train['stroke'] = y_train

df_majority = X_train[X_train.stroke==0]
df_minority = X_train[X_train.stroke==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=int(len(df_majority)/3),    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
X_train = pd.concat([df_majority, df_minority_upsampled])

y_train = X_train['stroke']
X_train.drop(['stroke'], axis = 1, inplace = True)

depth = [2,3,4]
leaf = [1]
for i in range(4):
    for j in depth:
        n_estimator = 70 + i*5
        grd = GradientBoostingClassifier(n_estimators=n_estimator,max_depth=j)
        grd.fit(X_train, y_train)
        y_pred_grd = grd.predict_proba(X_test)[:, 1]
        fpr_grd, tpr_grd, _ = roc_curve(y_test, y_pred_grd)
        key = str(n_estimator) + "_" + str(j)
        print ("GBC ", "N&Depth: ",key," Value: ",round(auc(fpr_grd, tpr_grd),3))
print ("Done")

C:\Users\shubham.gupta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


GBC  N&Depth:  70_2  Value:  0.847
GBC  N&Depth:  70_3  Value:  0.844
GBC  N&Depth:  70_4  Value:  0.838
GBC  N&Depth:  75_2  Value:  0.847
GBC  N&Depth:  75_3  Value:  0.845
GBC  N&Depth:  75_4  Value:  0.837
GBC  N&Depth:  80_2  Value:  0.847
GBC  N&Depth:  80_3  Value:  0.844
GBC  N&Depth:  80_4  Value:  0.837
GBC  N&Depth:  85_2  Value:  0.847
GBC  N&Depth:  85_3  Value:  0.845
GBC  N&Depth:  85_4  Value:  0.837
Done


In [52]:
test_id = test_data['id']

train_x['stroke'] = train_y

df_majority = train_x[train_x.stroke==0]
df_minority = train_x[train_x.stroke==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=int(len(df_majority)/40),    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
train_x = pd.concat([df_majority, df_minority_upsampled])

train_y = train_x['stroke']
train_x.drop(['stroke'], axis = 1, inplace = True)



grd = GradientBoostingClassifier(n_estimators=65, max_depth=2)
grd.fit(train_x, train_y)
y_pred_grd = grd.predict_proba(test_data)[:,1]
y_pred_grd1 = grd.predict(test_data)
print (round(100*np.mean(y_pred_grd1)))
print (sum(y_pred_grd1))
#pr_grd, tpr_grd, _ = roc_curve(y_test, y_pred_grd)

0.0
1


In [50]:
y_pred_grd

array([0.16711342, 0.18891896, 0.00356275, ..., 0.20090546, 0.02335519,
       0.01073629])

In [53]:
import csv
with open('Submission/model12.csv',"w",newline='') as f:
    writer = csv.writer(f)
    ps = ['id','stroke']
    writer.writerow(ps)
    for x,vc  in  zip(test_id,y_pred_grd):
            writer.writerow([x,vc])

In [33]:
len(y_pred_grd)

8680

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [70]:
n_estimator = 40
from sklearn.metrics import auc
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)


clf = RandomForestClassifier(max_depth=2, min_samples_leaf=0.05, n_estimators =n_estimator, random_state=0)
clf.fit(X_train, y_train)
#y_pred = clf.predict(valid)
y_pred_rf = clf.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_rf)
print ("RFC", auc(fpr_rf, tpr_rf))

X_train, X_train_lr, y_train, y_train_lr = train_test_split(X_train,
                                                            y_train,
                                                            test_size=0.2,random_state=42)


rf = RandomForestClassifier(max_depth=3, min_samples_leaf=0.05,  n_estimators=n_estimator)
rf_enc = OneHotEncoder()
rf_lm = LogisticRegression()
rf.fit(X_train, y_train)
rf_enc.fit(rf.apply(X_train))
rf_lm.fit(rf_enc.transform(rf.apply(X_train_lr)), y_train_lr)

y_pred_rf_lm = rf_lm.predict_proba(rf_enc.transform(rf.apply(X_test)))[:, 1]
fpr_rf_lm, tpr_rf_lm, _ = roc_curve(y_test, y_pred_rf_lm)
print ("RFC+LR", auc(fpr_rf_lm, tpr_rf_lm))




grd = GradientBoostingClassifier(n_estimators=n_estimator)
grd_enc = OneHotEncoder()
grd_lm = LogisticRegression()
grd.fit(X_train, y_train)
grd_enc.fit(grd.apply(X_train)[:, :, 0])
grd_lm.fit(grd_enc.transform(grd.apply(X_train_lr)[:, :, 0]), y_train_lr)

y_pred_grd_lm = grd_lm.predict_proba(
    grd_enc.transform(grd.apply(X_test)[:, :, 0]))[:, 1]
fpr_grd_lm, tpr_grd_lm, _ = roc_curve(y_test, y_pred_grd_lm)
print ("GBC+LR", auc(fpr_grd_lm, tpr_grd_lm))

# The gradient boosted model by itself
y_pred_grd = grd.predict_proba(X_test)[:, 1]
fpr_grd, tpr_grd, _ = roc_curve(y_test, y_pred_grd)
print ("GBC", auc(fpr_grd, tpr_grd))

RFC 0.8159951558773629
RFC+LR 0.7279843594041525
GBC+LR 0.7260095992954002
GBC 0.8128363417104185


GBC 0.8128800875500068
